# Python API for KiFMM-rs

We provide a full, object oriented, Python API for our Rust library. Below are a number of examples of how to achieve different functionality.

In [1]:
import numpy as np

from kifmm_py import (
    KiFmm,
    LaplaceKernel,
    SingleNodeTree,
    EvalType,
    BlasFieldTranslation,
    FftFieldTranslation,
)

## Basic Example

Here we set some random charge and particle data, and evaluate the FMM with a given kernel and field translation mode.

In [7]:
np.random.seed(0)

dim = 3
dtype = np.float32
ctype = np.complex64

# Set FMM Parameters
expansion_order = np.array([6], np.uint64)  # Single expansion order
n_vec = 1
n_crit = 150
# depth = 2
n_sources = 1000
n_targets = 2000

# Setup source/target/charge data in Fortran order
sources = np.random.rand(n_sources * dim).astype(dtype)
targets = np.random.rand(n_targets * dim).astype(dtype)
charges = np.random.rand(n_sources * n_vec).astype(dtype)

kernel = LaplaceKernel(
    dtype, EvalType.Value
)  # Evaluation mode computes either potentials, potentials + derivatives
field_translation = FftFieldTranslation(
    kernel, block_size=32
)  # Block size determines cache usage, leave as None for default
tree = SingleNodeTree(sources, targets, charges, n_crit=n_crit, prune_empty=True)

# Create FMM runtime object
fmm = KiFmm(expansion_order, tree, field_translation, timed=False)

In [8]:
fmm.fmm

<cdata 'struct FmmEvaluator *' 0x1029045c0>

In [13]:
# Run the FMM
fmm.evaluate()

# Clear the charges, re-attach new charges
new_charges = np.reshape(np.random.rand(n_sources * n_vec), (n_sources, n_vec)).astype(
    dtype
)
fmm.clear(new_charges)

# Can now re-run the FMM
fmm.evaluate()

## Querying Tree

## Evaluating Directly

## 3D Plot with MayaVi